In [1]:
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
import pickle
from sklearn.preprocessing import LabelEncoder 
import xgboost as xgb

2024-03-03 21:52:38.040697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 21:52:38.648744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-03 21:52:38.648797: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-03 21:52:38.648801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

tf.keras.utils.set_random_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import relevant datasets
train = pd.read_csv("trainsplitfinal.csv")
test = pd.read_csv("testsplitfinal.csv")
val = pd.read_csv("valsplitfinal.csv")

In [4]:
#shuffle the datasets before splitting into X and Y
train = train.sample(frac=1, random_state=42)
test = test.sample(frac=1, random_state=42)
val = val.sample(frac=1, random_state=42)

In [5]:
#make a combined test dataset
wholetest = pd.concat([test,val],axis = 0)
wholetest.shape

(189, 73)

In [6]:
train['labels']

265     L_Histidine
78      L_Glutamine
347     L_Glutamate
255        L_Lysine
327     L_Glutamine
           ...     
106     L_Glutamate
270    L_Methionine
348        L_Lysine
435     L_Glutamine
102       L_Alanine
Name: labels, Length: 439, dtype: object

In [7]:
encoder = LabelEncoder()
encoder.fit(train['labels'])
y_train= encoder.transform(train['labels'])
y_test = encoder.transform(wholetest['labels'])

In [8]:
modelwithoutweights = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
modelwithoutweights.fit(train.iloc[:,:72],y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [9]:
accuracy_score(y_test,modelwithoutweights.predict(wholetest.iloc[:,:72]))

0.26455026455026454

In [10]:
accuracy_score(y_train,modelwithoutweights.predict(train.iloc[:,:72]))

1.0

In [12]:
with open('xgbnoweights.pickle', 'wb') as f:
    pickle.dump(modelwithoutweights, f)